### 作業目的: 使用N-Gram模型預測文字

本次作業會使用[桃園市官網市政新聞](https://data.gov.tw/dataset/25891)來進行練習

### 載入所需的Libraries

In [1]:
import json
import re
from collections import Counter, namedtuple
import pandas as pd


### 載入資料

In [2]:
with open('./WebNews.json', 'r',encoding='utf8',errors='ignore') as f:
    ### <your code> ###
    news_data = json.load(f)
    
def clean_corpus(corpus):
    return re.sub(r'\W','',re.sub('<.*?>|&.*?;','',corpus))
    

### 進行資料清洗
觀察上面的資料，資料包含許多其他的資訊，我們需要將真正的新聞內文取出，並且對內文進行文字清洗。
請做以下的文字處理:

1. 取出新聞內文
2. 去除HTML Tags
3. 移除標點符號，只保留英文、數字、中文

In [3]:
#取出新聞內文
corpus_list = []### <your code> ###
for i in news_data:
    corpus_list.append(clean_corpus(i['detailcontent']))
#去除HTML Tags與標點符號(只保留英文、數字、中文)
#corpus_list = ### <your code> ###
len(corpus_list)

1000

### 建立N-Gram模型
N-Gram模型在計算詞機率時為(以Trigram為例)
$$
P(W_i|W_{i-1},W_{i-2}) = \frac{count(W_i,W_{i-1},W_{i-2})}{count(W_{i-1},W_{i-2})}
$$

舉例來說
$$
P(the|this,is) = \frac{count(this\ is\ the)}{count(this\ is)}
$$

### 使用N-Gram模型進行預測
這裡我們使用4 gram模型，也就是輸入三個字之後，第四個字出現的機率，並將輸出依據機率做排序

In [19]:
def n_gram_split(corpus,n):
    lst = [corpus[i:i+n] for i in range(len(corpus)) if i <= len(corpus)-n]
    return {i:lst.count(i) for i in lst}

def n_gram_prediction(n,corpus_list):
    
    globals()[f'n{n}'] = Counter()
    globals()[f'n{n-1}'] = Counter()
    for i in corpus_list:
        globals()[f'n{n-1}'] = eval(f'n{n-1}') + Counter(n_gram_split(i,n-1))
        globals()[f'n{n}'] = eval(f'n{n}') + Counter(n_gram_split(i,n))
        
    prob_dict = {k:eval(f'n{n}')[k]/eval(f'n{n-1}')[k[:-1]] for k in eval(f'n{n}').keys()}
    return prob_dict

n_4 = n_gram_prediction(4,corpus_list)

In [22]:
def input_string(prob_dict,string):
    return {k:prob_dict[k] for k in prob_dict.keys() if k[:-1] == string}

input_string(n_4,'鄭文燦')

{'鄭文燦今': 0.9795698924731183,
 '鄭文燦模': 0.002150537634408602,
 '鄭文燦市': 0.00967741935483871,
 '鄭文燦台': 0.001075268817204301,
 '鄭文燦率': 0.001075268817204301,
 '鄭文燦於': 0.001075268817204301,
 '鄭文燦表': 0.002150537634408602,
 '鄭文燦昨': 0.001075268817204301,
 '鄭文燦成': 0.001075268817204301,
 '鄭文燦回': 0.001075268817204301}